# Multi-Model Integration: Using Gemini, DeepSeek & Groq with OpenAI Agents

In [29]:
# Libraries and Imports
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel
from openai.types.responses import ResponseTextDeltaEvent
from openai import AsyncOpenAI
import sendgrid
import os
from sendgrid.helpers.mail import Email, To, Content, Mail

# Load environment variables
load_dotenv(override=True)

# Load API Keys and Verify
api_keys = {
    "openai": os.getenv('OPENAI_API_KEY'),
    "google": os.getenv('GEMINI_API_KEY'),
    "deepseek": os.getenv('DEEPSEEK_API_KEY'),
    "groq": os.getenv('GROQ_API_KEY')
}

for provider, key in api_keys.items():
    if key:
        print(f"{provider.title()} API Key loaded: {key[:10]}...")
    else:
        print(f"{provider.title()} API Key missing.")

Openai API Key loaded: sk-proj-oR...
Google API Key loaded: AIzaSyAd8c...
Deepseek API Key loaded: sk-ba892fd...
Groq API Key loaded: gsk_Le5WL7...


### Define Sales Agent Personas

In [30]:
# Personalized agent instructions
agent_styles = {
    "direct": "You write serious, clear, and professional sales emails for SOC2 compliance solutions.",
    "witty": "You write humorous, attention-grabbing emails for SOC2 compliance tools.",
    "concise": "You write minimal, punchy cold emails ideal for busy executives."
}

### Initialize Multi-Model Clients

In [31]:
# Base URLs for LLMs
MODEL_BASE_URLS = {
    "gemini": "https://generativelanguage.googleapis.com/v1beta/openai/",
    "deepseek": "https://api.deepseek.com/v1",
    "groq": "https://api.groq.com/openai/v1"
}

# Create Agent Clients
model_clients = {
    "gemini": AsyncOpenAI(base_url=MODEL_BASE_URLS["gemini"], api_key=api_keys["google"]),
    "deepseek": AsyncOpenAI(base_url=MODEL_BASE_URLS["deepseek"], api_key=api_keys["deepseek"]),
    "groq": AsyncOpenAI(base_url=MODEL_BASE_URLS["groq"], api_key=api_keys["groq"])
}

### Build Model-Specific Agents

In [32]:
# Create the Agents of LLMs
agent_models = {
    "gemini": OpenAIChatCompletionsModel("gemini-2.0-flash", model_clients["gemini"]),
    "deepseek": OpenAIChatCompletionsModel("deepseek-chat", model_clients["deepseek"]),
    "llama3": OpenAIChatCompletionsModel("llama-3.3-70b-versatile", model_clients["groq"]),
}

# Sales Agents - Multi LLM Model Based
sales_agents = [
    Agent(name="Gemini Agent", instructions=agent_styles["witty"], model=agent_models["gemini"]),
    Agent(name="DeepSeek Agent", instructions=agent_styles["direct"], model=agent_models["deepseek"]),
    Agent(name="Groq Agent", instructions=agent_styles["concise"], model=agent_models["llama3"])
]

# Convert agents to tools
email_task_description = "Generate a cold outreach email for a SaaS compliance product."
agent_tools = [
    agent.as_tool(tool_name=f"{agent.name.lower().replace(' ', '_')}_tool", tool_description=email_task_description)
    for agent in sales_agents
]

agent_tools

[FunctionTool(name='gemini_agent_tool', description='Generate a cold outreach email for a SaaS compliance product.', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'gemini_agent_tool_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10f9ae700>, strict_json_schema=True),
 FunctionTool(name='deepseek_agent_tool', description='Generate a cold outreach email for a SaaS compliance product.', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'deepseek_agent_tool_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10f8e7ec0>, strict_json_schema=True),
 FunctionTool(name='groq_agent_tool', description='Generate a cold outreach email for a SaaS compliance 

## Email Formatting & Sending Utilities

#### Tool: SendGrid Email Sender

In [33]:
# Function Tool to Send Email via SendGrid
@function_tool
def send_email(subject: str, html_body: str):
     """Send an HTML email with subject and body to the recipient list."""
     sg = sendgrid.SendGridAPIClient(api_key=os.environ.get("SENDGRID_API_KEY"))
     sender = Email("anshulc55@gmail.com")
     recipient = To("anshulc55@icloud.com")
     content = Content("text/html", html_body)
     email = Mail(sender, recipient, subject, content).get()
     response = sg.client.mail.send.post(request_body=email)
     return {"status": "Success" , "code": response.status_code}

#### Utility Agents

In [34]:
# Subject Line Generator
subject_agent = Agent(
    name="Subject Line Creator",
    instructions="Write catchy subject lines for cold outreach emails.",
    model="gpt-4o-mini"
)
subject_tool = subject_agent.as_tool("subject_writer", "Generate a catchy email subject line")

# HTML Formatter
html_agent = Agent(
    name="HTML Formatter",
    instructions="Convert plain text emails (with markdown) to clean HTML layouts.",
    model="gpt-4o-mini"
)
html_tool = html_agent.as_tool("html_converter", "Convert plain text email to HTML")

### Email Orchestration Agent

In [35]:
email_utilities = [subject_tool, html_tool, send_email]

email_formatter = Agent(
    name="Email Orchestrator",
    instructions="Take a plain email body, generate a subject, convert to HTML, and send it.",
    tools=email_utilities,
    model="gpt-4o-mini",
    handoff_description="Format and send the final sales email"
)

### Sales Manager Agent: Multi-Model Integration

In [17]:
manager_instructions = """You are a sales manager for TechLink. Your job is to:
1. Try all available sales agents at least once.
2. Evaluate the quality of their emails.
3. Choose the most effective email.
4. Hand off to the Email Orchestrator agent for formatting and sending."""

sales_manager = Agent(
    name="Sales Manager",
    instructions=manager_instructions,
    tools=agent_tools,
    handoffs=[email_formatter],
    model="gpt-4o-mini"
)

## Execution and Tracing

In [46]:
message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Multi-Model Sales Email Automation"):
    result = await Runner.run(sales_manager, message)

### Check out the trace:

https://platform.openai.com/traces

# Example: Guardrail for Name Detection
Prevent the sales agent from running if the input message includes a personal name (e.g., "Alice", "John").

In [36]:
# Define the Structured Output Schema
from pydantic import BaseModel

class NameCheckOutput(BaseModel):
    is_name_in_message: bool
    name: str

## - Schema enforces that the output from the name-detection agent must include: is_name_in_message, name

In [37]:
# Create the Guardrail Agent
guardrail_agent = Agent(
    name="Name Validator",
    instructions="Analyze the input to determine if a personal name is present.",
    output_type=NameCheckOutput,
    model="gpt-4o-mini"
)

## - This agent uses the structured output to enforce strict control over its response format.

#### Define the Input Guardrail Function

In [38]:
from agents import input_guardrail, GuardrailFunctionOutput

@input_guardrail  # Registers this function as a pre-check.
async def block_if_personal_name_detected(ctx, agent, message):
    result = await Runner.run(guardrail_agent, message, context=ctx.context)
    is_triggered =  result.final_output.is_name_in_message
    return GuardrailFunctionOutput(
        output_info={"detected_name_info": result.final_output},  # Provides insight into why the guardrail was triggered.
        tripwire_triggered=is_triggered   # tripwire_triggered=True: Automatically stops the main agent.
    )

# block_if_personal_name_detected

#### Attach Guardrail to the Sales Manager Agent

In [39]:
manager_instructions = """You are a sales manager for TechLink. Your job is to:
1. Try all available sales agents at least once.
2. Evaluate the quality of their emails.
3. Choose the most effective email.
4. Hand off to the Email Orchestrator agent for formatting and sending."""

professional_sales_manager = Agent(
    name="Professional Sales Manager",
    instructions=manager_instructions,
    tools=agent_tools,
    handoffs=[email_formatter],
    model="gpt-4o-mini",
    input_guardrails=[block_if_personal_name_detected]
)

#### Execute with and without Violations

In [40]:
# Execute with Voilation
message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Sales Protected Email - With Name"):
    result = await Runner.run(professional_sales_manager, message)

InputGuardrailTripwireTriggered: Guardrail InputGuardrail triggered tripwire

In [41]:
# Execute without Voilation
message = "Send out a cold sales email addressed to Dear CEO from TechLink Head of Business"

with trace("Sales Protected Email - Without Name"):
    result = await Runner.run(professional_sales_manager, message)

### Check out the trace:

https://platform.openai.com/traces